# Perplejidad

In [1]:
import pickle
from collections import Counter
import math

#### Carga de los archivos .plk que tienen las probabilidades de cada n-grama

In [2]:
# news unigrams
news_unigrams_file = open("20N_unigrams.pkl","rb")
news_unigrams_pkl = pickle.load(news_unigrams_file)

# blogs unigrams
blogs_unigrams_file = open("BAC_unigrams.pkl","rb")
blogs_unigrams_pkl = pickle.load(blogs_unigrams_file)

# news bigrams
news_bigrams_file = open("20N_bigrams.pkl","rb")
news_bigrams_pkl = pickle.load(news_bigrams_file)

# blogs bigrams
blogs_bigrams_file = open("BAC_bigrams.pkl","rb")
blogs_bigrams_pkl = pickle.load(blogs_bigrams_file)

#### Creacion de diccionarios de probabilidades

In [3]:
# news unigrams probabilities
news_probs_unigrams = {sample[0]:news_unigrams_pkl.prob(sample[0]) for sample in news_unigrams_pkl.samples()}

# blogs unigrams probabilities
blogs_probs_unigrams = {sample[0]:blogs_unigrams_pkl.prob(sample[0]) for sample in blogs_unigrams_pkl.samples()}

# news bigrams probabilities
news_probs_bigrams = {sample:news_bigrams_pkl.prob(sample) for sample in news_bigrams_pkl.samples()}

# blogs bigrams probabilities
blogs_probs_bigrams = {sample:blogs_bigrams_pkl.prob(sample) for sample in blogs_bigrams_pkl.samples()}


#### Lectura de los datasets de prueba y creacion de counters con las palabras

In [4]:
# Se cargan los archivos de test
with open('20N_GROUP_testing.txt', 'r', encoding='utf-8', errors='ignore') as file:
    news_texts_test = file.read()

# Vamos a separar el corpus de train de 20N en sentencias
news_texts_train = news_texts_test.split('. ')
print('Cantidad de oraciones en 20N_GROUP_testing: ', len(news_texts_test))

# Vamos a recorrer cada sentencia y separarla en palabras para agregarlas a un diccionario de frecuencias
news_word_counts_test = Counter()
for sentence in news_texts_train:
    # Se separa la sentencia en palabras
    sentence = sentence.split(' ')
    # Si el ultimo caracter de una palabra no es una letra, se quita
    for i in range(len(sentence)):
        if sentence[i] != "" and not sentence[i][-1].isalpha():
            sentence[i] = sentence[i][:-1]
    # Se agregan las palabras al diccionario de frecuencias
    news_word_counts_test.update(sentence)

Cantidad de oraciones en 20N_GROUP_testing:  6542527


In [5]:
# Se cargan los archivos de test
with open('BAC_GROUP_testing.txt', 'r', encoding='utf-8', errors='ignore') as file:
    blogs_texts_test = file.read()

# Vamos a separar el corpus de train de 20N en sentencias
blogs_texts_train = blogs_texts_test.split('. ')
print('Cantidad de oraciones en BAC_GROUP_testing: ', len(blogs_texts_test))

# Vamos a recorrer cada sentencia y separarla en palabras para agregarlas a un diccionario de frecuencias
blogs_word_counts_test = Counter()
for sentence in blogs_texts_train:
    # Se separa la sentencia en palabras
    sentence = sentence.split(' ')
    # Si el ultimo caracter de una palabra no es una letra, se quita
    for i in range(len(sentence)):
        if sentence[i] != "" and not sentence[i][-1].isalpha():
            sentence[i] = sentence[i][:-1]
    # Se agregan las palabras al diccionario de frecuencias
    blogs_word_counts_test.update(sentence)

Cantidad de oraciones en BAC_GROUP_testing:  151031140


#### Funciones para calcular perplejidad

In [6]:
def unigram_perplexity(test_data,probability):
    # Inicializa la entropía cruzada y el contador de palabras
    cross_entropy = 0.0
    word_count = 0

    for word in test_data:
        if word in probability:
            word_prob = probability[word]
            cross_entropy += math.log10(word_prob)
        else:
            word_prob = probability["<UNK>"]
            cross_entropy += math.log10(word_prob)
        word_count += 1

    # Calcula la entropía cruzada promedio (dividida por el número de palabras)
    cross_entropy /= word_count

    # Calcula la perplejidad como 2 elevado a la entropía cruzada
    perplexity = math.pow(2, -cross_entropy)

    return perplexity

In [13]:
def bigram_perplexity(test_data, probability):
    prev = '<s>'
    # Inicializa la entropía cruzada y el contador de pares de palabras
    cross_entropy = 0.0
    pair_count = 0

    for w in test_data:
        if (prev, w) in probability:
            word_pair_prob = probability[(prev, w)]
            cross_entropy += math.log10(word_pair_prob)
            pair_count += 1
        prev = w

    # Calcula la entropía cruzada promedio (dividida por el número de pares de palabras)
    cross_entropy /= pair_count

    # Calcula la perplejidad como 2 elevado a la entropía cruzada
    perplexity = math.pow(2, cross_entropy)
    return perplexity


#### Calculos de perplejidad

In [15]:
news_perx_unigrams = round(unigram_perplexity(news_word_counts_test,news_probs_unigrams),4)
blogs_perx_unigrams = round(unigram_perplexity(blogs_word_counts_test,blogs_probs_unigrams),4)
print(f"Perplejidad unigramas news = {str(news_perx_unigrams)}")
print(f"Perplejidad unigramas blogs = {str(blogs_perx_unigrams)}")

Perplejidad unigramas news = 106.7501
Perplejidad unigramas blogs = 280.4741


In [16]:
news_perx_bigrams = round(bigram_perplexity(news_word_counts_test, news_probs_bigrams),4)
blogs_perx_bigrams = round(bigram_perplexity(blogs_word_counts_test, blogs_probs_bigrams),4)
print(f"Perplejidad bigramas news = {str(news_perx_bigrams)}")
print(f"Perplejidad bigramas blogs = {str(blogs_perx_bigrams)}")


Perplejidad bigramas news = 0.0133
Perplejidad bigramas blogs = 0.0051
